In [ ]:
!pip install datasets transformers --q

     |████████████████████████████████| 441 kB 4.6 MB/s 
     |████████████████████████████████| 5.5 MB 80.7 MB/s 
     |████████████████████████████████| 115 kB 85.7 MB/s 
     |████████████████████████████████| 212 kB 85.4 MB/s 
     |████████████████████████████████| 163 kB 88.9 MB/s 
     |████████████████████████████████| 95 kB 6.0 MB/s 
     |████████████████████████████████| 127 kB 90.7 MB/s 
     |████████████████████████████████| 7.6 MB 74.6 MB/s 
     |████████████████████████████████| 115 kB 91.9 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# HuggingFace
from datasets import load_dataset
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
import torch

# load dataset from huggingface
midjourney = load_dataset('succinctly/midjourney-prompts')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPT2LMHeadModel.from_pretrained('/content/drive/Shareddrives/NLP_Project/checkpoint-82400').to(device)
tokenizer = GPT2Tokenizer.from_pretrained(
    'gpt2', 
    bos_token='<|startoftext|>', 
    eos_token='<|endoftext|>', 
    pad_token='<|pad|>',
    return_tensors='pt'
)
def tokenize_func(item):
    return tokenizer(item['text'], truncation=True, padding=True)

tokenized_midjourney = midjourney.map(tokenize_func, batched=True)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/succinctly___parquet/succinctly--midjourney-prompts-a6d12e8889ba9868/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


...

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/222 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

In [ ]:
tokenized_midjourney

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 221743
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 12318
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 12320
    })
})

In [ ]:
chef = pipeline(
    'text-generation',
    model=model, 
    tokenizer=tokenizer,
    config={'max_length':3000},
    device=0
)

In [ ]:
chef('Elon Musk in Twitter')[0]['generated_text']

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Elon Musk in Twitter, album Cover, hyper-realistic, insanely detailed and intricate, dark, 8k, cinematic --ar 5:3 --uplight --iw 1.8 --stop 85 --uplight --no text --'

In [ ]:
result = chef('A beautiful girl in the classroom')[0]['generated_text']
result

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


'A beautiful girl in the classroom, Studio Ghibli --ar 16:9 --uplight --stop 92 --hd --test --hd --ar 16:9 --test --hd --ar 16:9 --test --hd --test --'

In [ ]:
chef('Elon Musk Twitter')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Elon Musk Twitter Facebook Instagram Instagram PinterestY2K Uncertainty conjoined starving skeletal humanoids descending a staircase in the style of constantin brancusi and ernst haeckel :: cinematic :: realistic :: intricate :: detailed :: 6'

In [ ]:
chef('A photo of a dog as a superhero')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'A photo of a dog as a superhero in the Marvel Cinematic movie Rogue. :: pixar. :: photo. Marvel style movie artwork. :: Marvel Comics style look. :: Marvel Comics colors. :: --ar 9:16 --uplight'

In [ ]:
chef('medieval big ship at a furious sea at night.')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'medieval big ship at a furious sea at night. dark sea with light leaks on the windows, water splashes on the ground, painted by craig mullins mucha, high detail, lifelike, high octane, moody,'

In [ ]:
import math
import torch
from tqdm import tqdm
import numpy as np

def score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    loss=model(tensor_input.to(device), labels=tensor_input.to(device))['loss']
    # print(loss['loss'])
    return loss, math.exp(loss)

testset=tokenized_midjourney['test']['text']
trainset=tokenized_midjourney['train']['text']
# a=['there is a book on the desk',
#                 'there is a plane on the desk',
#                         'there is a book in the desk']
perplexity=[]
loss = []
for i,sentence in enumerate(tqdm(trainset)):
  # if i > 37:
  #   break
  # if i == 36:
  #   print(sentence)
    # perplexity.append(score(sentence))
  try:
    l, p = score(sentence)
    loss.append(l)
    perplexity.append(p)
  except:
    loss.append(math.nan)
    perplexity.append(math.nan)

print(perplexity)
np.save("train_perplexity.npy",np.array(perplexity))

100%|██████████| 221743/221743 [1:00:14<00:00, 61.34it/s]

[nan, nan, 1.9978743213029608, 28.828256077711817, 1.0486303636181398, 1.2219521709708927, 1.1381857407865534, 19.164240786238487, 4.540021303854353, 1.0479504512186932, 1.0131299853403117, 2.2003357922035574, 5.342242296880471, nan, 11.285160323453344, 10.14847498355473, 2.074422255739614, 1.0131299853403117, 3465.839304785757, 1.0131299853403117, 1.2078745734015823, 1.7857582302419703, 2.6708883866805304, 1.0131299853403117, 1.2011632195030346, 1.0131863357627133, 1.1538220476727998, 1.1026859129769684, 2.755052657527774, 3.6752308736205865, 2534.3555047100663, 5.342242296880471, 9.098947354120977, 3.201582655824772, 3.7249527911629645, nan, 1.0714333015531303, 1.2557446451579053, 1.0131299853403117, 1.1026859129769684, 4.813399077279982, 1.6522051313178876, 2.921834704425688, 2.1839815843615, 1.204945578752641, 35.61196459133034, 14.007245986120523, 1.571154341207035, 5.388590524987163, 1.1026859129769684, 1.1026859129769684, nan, 3.920452197260937, nan, nan, 105.48591555643192, 1.0

In [ ]:
loss[2].to('cpu').detach().numpy()

TypeError: ignored

In [ ]:
new_loss = []
new_perplexity = []
for i in range(len(loss)):
  if math.isnan(loss[i]):
    continue
  else:
    new_loss.append(loss[i].to('cpu').detach().numpy())
  if math.isnan(perplexity[i]):
    continue
  else:
    new_perplexity.append(perplexity[i])

In [ ]:
np.mean(new_loss)
np.exp(np.mean(new_loss))

3.7203782

In [ ]:
perplexity_new = [item for item in perplexity if (math.isnan(item)) == False]
len(perplexity_new)
import matplotlib.pyplot as plt
np.percentile(perplexity_new, 95)

70.40269300521257

In [ ]:
np.nanargmax(perplexity)

18

In [ ]:
testset[9076]

'draw autism'

In [ ]:
tokenize_input = tokenizer.tokenize(testset[6])
tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)]).to(device)
loss=model(tensor_input, labels=tensor_input)['loss']
print(testset[6])
print(loss)
print(chef(testset[6])[0]['generated_text'])

RuntimeError: ignored

In [ ]:
logits = model(tensor_input, labels=tensor_input)["logits"]
model(tensor_input, labels=tensor_input)

In [ ]:
t0 = torch.argmax(torch.nn.functional.softmax(logits[0][0]))
t1 = torch.argmax(torch.nn.functional.softmax(logits[0][1]))
tokenizer.decode([t1])
# print(logits[0][1])

In [ ]:
trainset=tokenized_midjourney['train']['text']
prompt_length = []
for t in trainset:
  prompt_length.append(len(t.split(" ")))
plt.hist(prompt_length, bins=20)